In [8]:
# !pip install pandas
import pandas as pd

#data = pd.read_csv('./data/IMMREP/train/all_peptides_redundancy_reduced.csv')
data = pd.read_csv('./data/nettcr_2_2_limited_dataset.csv')
print(data.head(3))


   index      A1       A2              A3      B1       B2              B3  \
0      0   KALYS  LLKGGEQ  GTEIGGGTSYGKLT   MNHEY   SMNVEV        ASGTETQY   
1      1  DRGSQS   IYSNGD      AVNPANARLM  DFQATT  SNEGSKA    SARWGGGTDTQY   
2      3  DRGSQS   IYSNGD      AVTPGTYKYI   LGHDT   YNNKEL  ASSPGTSIFVAEQY   

     peptide       allele           origin  binder  partition  \
0  SPRWYFYYL  HLA-B*07:02  peptide_swapped       0          2   
1  GILGFVFTL  HLA-A*02:01  peptide_swapped       0          3   
2  KLGGALQAK  HLA-A*03:01  peptide_swapped       0          0   

  original_peptide  original_index   10x reference  
0        KLGGALQAK            2627  True  negative  
1       ELAGIGILTV            3820  True  negative  
2       ELAGIGILTV            5933  True  negative  


In [9]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34352 entries, 0 to 34351
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   index             34352 non-null  int64 
 1   A1                34352 non-null  object
 2   A2                34352 non-null  object
 3   A3                34352 non-null  object
 4   B1                34352 non-null  object
 5   B2                34352 non-null  object
 6   B3                34352 non-null  object
 7   peptide           34352 non-null  object
 8   allele            34352 non-null  object
 9   origin            34352 non-null  object
 10  binder            34352 non-null  int64 
 11  partition         34352 non-null  int64 
 12  original_peptide  34352 non-null  object
 13  original_index    34352 non-null  int64 
 14  10x               34352 non-null  bool  
 15  reference         33949 non-null  object
dtypes: bool(1), int64(4), object(11)
memory usage: 4.0+ MB
Non

In [10]:
print(data.isnull().sum())

index                 0
A1                    0
A2                    0
A3                    0
B1                    0
B2                    0
B3                    0
peptide               0
allele                0
origin                0
binder                0
partition             0
original_peptide      0
original_index        0
10x                   0
reference           403
dtype: int64


In [19]:
import os
import subprocess
from sklearn.model_selection import train_test_split, KFold

train_val, test = train_test_split(data, test_size=0.2, random_state=42)  # 20% data as test set

# 5-fold cross validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# train_val data split
for fold, (train_idx, val_idx) in enumerate(kf.split(data)):
    # create output directory
    outdir = f'results/fold_{fold}'
    if not os.path.exists(outdir):
        os.makedirs(outdir)

    # split data
    train_data = data.iloc[train_idx]
    val_data = data.iloc[val_idx]

    # save data
    train_file = f'{outdir}/train_fold_{fold}.csv'
    val_file = f'{outdir}/val_fold_{fold}.csv'
    train_data.to_csv(train_file, index=False)
    val_data.to_csv(val_file, index=False)

    # train model
    cmd = f"python src/train_nettcr_2_2_pan.py --train_data {train_file} " \
          f"--val_data {val_file} --outdir {outdir} " \
          f"--model_name model_1204_fold_{fold} --epochs 200 " \
          f"--batch_size 64 --learning_rate 0.001 --dropout_rate 0.6 " \
          f"--patience 100 --verbose 1"

    # 执行命令
    subprocess.run(cmd, shell=True)


2024-12-04 22:58:44.565826: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/200
86/86 [==============================] - 4s 15ms/step - loss: 0.4862 - auc_01: 0.5022 - auc: 0.5092 - val_loss: 0.4538 - val_auc_01: 0.5162 - val_auc: 0.5519
Epoch 2/200
86/86 [==============================] - 1s 9ms/step - loss: 0.4477 - auc_01: 0.5127 - auc: 0.5132 - val_loss: 0.4524 - val_auc_01: 0.5093 - val_auc: 0.5497
Epoch 3/200
86/86 [==============================] - 1s 10ms/step - loss: 0.4473 - auc_01: 0.5078 - auc: 0.5066 - val_loss: 0.4529 - val_auc_01: 0.5035 - val_auc: 0.5335
Epoch 4/200
86/86 [==============================] - 1s 10ms/step - loss: 0.4477 - auc_01: 0.5048 - auc: 0.5004 - val_loss: 0.4548 - val_auc_01: 0.5180 - val_auc: 0.5297
Epoch 5/200
86/86 [==============================] - 1s 9ms/step - loss: 0.4472 - auc_01: 0.5102 - auc: 0.5051 - val_loss: 0.4525 - val_auc_01: 0.5161 - val_auc: 0.5462
Epoch 6/200
86/86 [==============================] - 1s 13ms/step - loss: 0.4457 - auc_01: 0.5115 - auc: 0.5225 - val_loss: 0.4500 - val_auc_01: 0.5168 

2024-12-04 23:02:51.949769: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2024-12-04 23:02:51.949797: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2024-12-04 23:02:51.950801: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/0f/qywr9xws0sx43t5v5n2ttn540000gn/T/tmp5hibh_6n
2024-12-04 23:02:51.959945: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2024-12-04 23:02:51.959957: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/0f/qywr9xws0sx43t5v5n2ttn540000gn/T/tmp5hibh_6n
2024-12-04 23:02:51.987309: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2024-12-04 23:02:51.995711: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2024-12-04 23:02:52.245475: I tensorflow/cc/saved_model/loader.

Figure(1500x1000)


2024-12-04 23:02:59.628083: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


...

Figure(1500x1000)


2024-12-04 23:15:43.496045: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/200
86/86 [==============================] - 6s 28ms/step - loss: 0.4772 - auc_01: 0.5099 - auc: 0.4864 - val_loss: 0.4780 - val_auc_01: 0.5109 - val_auc: 0.4819
Epoch 2/200
86/86 [==============================] - 2s 23ms/step - loss: 0.4439 - auc_01: 0.5032 - auc: 0.5094 - val_loss: 0.4783 - val_auc_01: 0.5092 - val_auc: 0.4524
Epoch 3/200
86/86 [==============================] - 1s 15ms/step - loss: 0.4428 - auc_01: 0.5125 - auc: 0.5106 - val_loss: 0.4758 - val_auc_01: 0.5089 - val_auc: 0.4537
Epoch 4/200
86/86 [==============================] - 1s 14ms/step - loss: 0.4429 - auc_01: 0.5164 - auc: 0.5077 - val_loss: 0.4717 - val_auc_01: 0.5105 - val_auc: 0.4471
Epoch 5/200
86/86 [==============================] - 1s 15ms/step - loss: 0.4430 - auc_01: 0.5122 - auc: 0.5069 - val_loss: 0.4806 - val_auc_01: 0.5123 - val_auc: 0.4676
Epoch 6/200
86/86 [==============================] - 1s 14ms/step - loss: 0.4413 - auc_01: 0.5123 - auc: 0.5256 - val_loss: 0.4773 - val_auc_01: 0.515

2024-12-04 23:19:45.193706: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2024-12-04 23:19:45.193730: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2024-12-04 23:19:45.194317: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/0f/qywr9xws0sx43t5v5n2ttn540000gn/T/tmpoqlw5kcp
2024-12-04 23:19:45.203041: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2024-12-04 23:19:45.203054: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/0f/qywr9xws0sx43t5v5n2ttn540000gn/T/tmpoqlw5kcp
2024-12-04 23:19:45.230716: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2024-12-04 23:19:45.238878: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2024-12-04 23:19:45.481385: I tensorflow/cc/saved_model/loader.

Figure(1500x1000)


In [20]:

# predict
test_file = f'{outdir}/test.csv'
test.to_csv(test_file, index=False)

for fold in range(5):
    outdir = f'results/fold_{fold}'
    cmd = f"python src/predict.py --test_data {test_file} --outdir {outdir} " \
          f"--model_name model_1204_fold_{fold} --model_type pan"
    subprocess.run(cmd, shell=True)

INFO: Initialized TensorFlow Lite runtime.
INFO: Applying 1 TensorFlow Lite delegate(s) lazily.


0.073 seconds


INFO: Initialized TensorFlow Lite runtime.
INFO: Applying 1 TensorFlow Lite delegate(s) lazily.


0.064 seconds


INFO: Initialized TensorFlow Lite runtime.
INFO: Applying 1 TensorFlow Lite delegate(s) lazily.


0.072 seconds


INFO: Initialized TensorFlow Lite runtime.
INFO: Applying 1 TensorFlow Lite delegate(s) lazily.


0.074 seconds


INFO: Initialized TensorFlow Lite runtime.
INFO: Applying 1 TensorFlow Lite delegate(s) lazily.


0.064 seconds


In [26]:
import pandas as pd
from sklearn.metrics import roc_auc_score

ds = 4
file_path = f'results/fold_{ds}/model_1204_fold_{ds}_prediction.csv'
data = pd.read_csv(file_path)

df = pd.DataFrame({
    'binder': data['binder'],  # 
    'prediction': data['prediction']  # 
})

# 
print(df.head())

# y_true and y_scores
y_true = df['binder']
y_scores = df['prediction']


from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve, auc, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

# full auc
full_auc = roc_auc_score(y_true, y_scores)
print("Full AUC:", full_auc)

# auc 0.1
auc_01 = roc_auc_score(y_true, y_scores, max_fpr = 0.1)
print("AUC 0.1:", auc_01)

# precision and recall
precision = precision_score(y_true, y_scores >= 0.5)  # threshold 0.5
recall = recall_score(y_true, y_scores >= 0.5)
print("Precision:", precision)
print("Recall:", recall)

# f1 score
f1 = f1_score(y_true, y_scores >= 0.5)
print("F1 Score:", f1)

# specificity
tn, fp, fn, tp = confusion_matrix(y_true, y_scores >= 0.5).ravel()
specificity = tn / (tn + fp)
print("Specificity:", specificity)

# accuracy
accuracy = accuracy_score(y_true, y_scores >= 0.5)
print("Accuracy:", accuracy)


   binder  prediction
0       0    0.009128
1       0    0.024446
2       0    0.011468
3       0    0.152820
4       0    0.000428
Full AUC: 0.9939965432325364
AUC 0.1: 0.9773768496402757
Precision: 0.9615384615384616
Recall: 0.8771929824561403
F1 Score: 0.9174311926605504
Specificity: 0.993025283347864
Accuracy: 0.9738181818181818


In [27]:
import pandas as pd
from sklearn.metrics import roc_auc_score

def calculate_metrics(num_folds):
    
    results = pd.DataFrame(columns=['Fold', 'AUC', 'AUC_0.1'])

    for fold in range(num_folds):
        file_path = f'results/fold_{fold}/model_1204_fold_{fold}_prediction.csv'
        data = pd.read_csv(file_path)

        df = pd.DataFrame({
            'binder': data['binder'],  
            'prediction': data['prediction']  
        })

        # 
        y_true = df['binder']
        y_scores = df['prediction']

        # 
        full_auc = roc_auc_score(y_true, y_scores)
        print(f"Fold {fold} Full AUC:", full_auc)

        # AUC 0.1
        #fpr, tpr, thresholds = roc_curve(y_true, y_scores)
        # auc_01 = auc(fpr[fpr <= 0.1], tpr[fpr <= 0.1])
        auc_01 = roc_auc_score(y_true, y_scores, max_fpr = 0.1)
        print(f"Fold {fold} AUC 0.1:", auc_01)

        # 
        results = results.append({'Fold': fold, 'AUC': full_auc, 'AUC_0.1': auc_01}, ignore_index=True)

    return results

# 
num_folds = 5
results_df = calculate_metrics(num_folds)
print(results_df)


Fold 0 Full AUC: 0.8181985041068232
Fold 0 AUC 0.1: 0.6901829897093948
Fold 1 Full AUC: 0.9898935437984674
Fold 1 AUC 0.1: 0.9581086313728616
Fold 2 Full AUC: 0.9988681380871534
Fold 2 AUC 0.1: 0.9940428320376493
Fold 3 Full AUC: 0.99890637666529
Fold 3 AUC 0.1: 0.9942440877120531
Fold 4 Full AUC: 0.9939965432325364
Fold 4 AUC 0.1: 0.9773768496402757
   Fold       AUC   AUC_0.1
0   0.0  0.818199  0.690183
1   1.0  0.989894  0.958109
2   2.0  0.998868  0.994043
3   3.0  0.998906  0.994244
4   4.0  0.993997  0.977377


/var/folders/0f/qywr9xws0sx43t5v5n2ttn540000gn/T/ipykernel_25977/54992578.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'Fold': fold, 'AUC': full_auc, 'AUC_0.1': auc_01}, ignore_index=True)
/var/folders/0f/qywr9xws0sx43t5v5n2ttn540000gn/T/ipykernel_25977/54992578.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'Fold': fold, 'AUC': full_auc, 'AUC_0.1': auc_01}, ignore_index=True)
/var/folders/0f/qywr9xws0sx43t5v5n2ttn540000gn/T/ipykernel_25977/54992578.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'Fold': fold, 'AUC': full_auc, 'AUC_0.1': auc_01}, ignore_index=True)
/var/folders/0f/qywr9xws0sx43t5v5n2ttn540000gn/T/ipykernel_25977